In [1]:
import matplotlib
matplotlib.use('TkAgg')  # or 'Qt5Agg'
import numpy as np
import simpy
import seaborn
import matplotlib.pyplot as plt
import random
import pandas as pd

In [2]:
class Pacjent:
    def __init__(self, id):
        self.day = 1
        self.id = id
        self.room = None
        self.arrival_time = None
        self.service_start_time = None
        self.service_end_time = None

In [3]:
class Gabinet:
    def __init__(self, id):
        self.id = id
        self.patients_served = 0

In [4]:
class Clinic:
    def __init__(self, number_of_rooms, service_time, queue_array=[], lambda_per_hour=6/60, seed=None, sim_time=120):
        self.curr_patient_id = 1
        self.env = simpy.Environment()
        self.service_time = service_time
        self.lambda_per_hour = lambda_per_hour
        self.seed = seed
        self.list_rooms = [Gabinet(id=i + 1) for i in range(number_of_rooms)]
        self.rooms = simpy.Store(self.env, capacity=number_of_rooms)
        self.sim_time = sim_time
        self.queue_array = queue_array
        for room in self.list_rooms:
            self.rooms.put(room)
        self.processed_patients = []

In [ ]:
def czas(self, time_in_minutes):
    hours = 8+time_in_minutes//60
    minutes = time_in_minutes%60
    if len(str(int(minutes)))==1:
        return f"{int(hours)}:0{int(minutes)}"
    return f"{int(hours)}:{int(minutes)}"
Clinic.czas = czas

In [6]:
def time_between_new_patients(self):
    if self.seed:
        np.random.seed(self.seed)
    return np.random.exponential(1/self.lambda_per_hour)
Clinic.time_between_new_patients = time_between_new_patients

In [7]:
def generate_patients(self):
    while True:
        if self.env.now >= self.sim_time-self.service_time: # Pacjenci nie przychodzą {service_time} przed zamknięciem
            break
        patient = Pacjent(id=self.curr_patient_id)
        patient.arrival_time = self.env.now
        print(f"Czas {self.czas(np.trunc(self.env.now))}: Pacjent {patient.id} przybył do kliniki")
        self.env.process(self.serve_patient(patient))
        self.curr_patient_id += 1
        yield self.env.timeout(self.time_between_new_patients())
Clinic.generate_patients = generate_patients

In [8]:
def serve_patient(self, patient): 
    room = yield self.rooms.get()
    self.queue_array.append(patient.id)
    if self.env.now >= self.sim_time-self.service_time: # Nie zdążyło obsłużyć pacjenta przed zamknięciem
        print(f"Czas {self.czas(np.trunc(self.env.now))}: Pacjent {patient.id} - nie zdążył zostać obsłużony przed zamknięciem kliniki")
        yield self.rooms.put(room)
        return
    patient.service_start_time = self.env.now
    print(f"Czas {self.czas(np.trunc(self.env.now))}: Pacjent {patient.id} wchodzi do gabinetu {room.id} ")
    self.queue_array.remove(patient.id)
    yield self.env.timeout(self.service_time)

    patient.service_end_time = self.env.now
    room.patients_served += 1
    print(f"Czas {self.czas(np.trunc(self.env.now))}: Pacjent {patient.id} wychodzi z gabinetu {room.id}, czekal od {self.czas(np.trunc(patient.arrival_time))} do {self.czas(np.trunc(patient.service_start_time))}")
    patient.room = room.id
    self.processed_patients.append(patient)
    yield self.rooms.put(room)
Clinic.serve_patient = serve_patient

In [9]:
def run(self, day=1):
    self.env.process(self.generate_patients())
    self.env.run(until=self.sim_time)
    df = pd.DataFrame([
    {
        "day": day,
        "id": p.id,
        "room": p.room,
        "arrival_time": p.arrival_time,
        "service_start_time": p.service_start_time,
        "service_end_time": p.service_end_time,
        "waiting_time": p.service_start_time - p.arrival_time
    }
    for p in self.processed_patients
    ])
    return df
Clinic.run = run

In [10]:
def reset(self):
    self.env = simpy.Environment()
    self.curr_patient_id = 1
    self.list_rooms = [Gabinet(id=i + 1) for i in range(len(self.list_rooms))]
    self.rooms = simpy.Store(self.env, capacity=len(self.list_rooms))
    for room in self.list_rooms:
        self.rooms.put(room)
    self.processed_patients = []
Clinic.reset = reset

In [11]:
def run_multiple_times(self, n):
    multiple_df = pd.DataFrame()
    for i in range(1, n+1):
        print(f"-----------{i}-----------")
        self.reset()
        single_df = self.run(i)
        multiple_df = pd.concat([multiple_df, single_df], ignore_index=True)
    return multiple_df
Clinic.run_multiple_times = run_multiple_times

In [12]:
clinic = Clinic(number_of_rooms=2, service_time=15, sim_time=120, lambda_per_hour=6/60)

In [13]:
clinic = Clinic(number_of_rooms=2, service_time=15, sim_time=120, lambda_per_hour=6/60)

In [14]:
# single_df = clinic.run(day=1)
df = clinic.run(day=1)

Czas 8:00: Pacjent 1 przybył do kliniki
Czas 8:00: Pacjent 1 wchodzi do gabinetu 1 
Czas 8:07: Pacjent 2 przybył do kliniki
Czas 8:07: Pacjent 2 wchodzi do gabinetu 2 
Czas 8:15: Pacjent 1 wychodzi z gabinetu 1, czekal od 8:00 do 8:00
Czas 8:22: Pacjent 2 wychodzi z gabinetu 2, czekal od 8:07 do 8:07
Czas 8:23: Pacjent 3 przybył do kliniki
Czas 8:23: Pacjent 3 wchodzi do gabinetu 1 
Czas 8:37: Pacjent 4 przybył do kliniki
Czas 8:37: Pacjent 4 wchodzi do gabinetu 2 
Czas 8:38: Pacjent 3 wychodzi z gabinetu 1, czekal od 8:23 do 8:23
Czas 8:43: Pacjent 5 przybył do kliniki
Czas 8:43: Pacjent 5 wchodzi do gabinetu 1 
Czas 8:44: Pacjent 6 przybył do kliniki
Czas 8:52: Pacjent 4 wychodzi z gabinetu 2, czekal od 8:37 do 8:37
Czas 8:52: Pacjent 6 wchodzi do gabinetu 2 
Czas 8:57: Pacjent 7 przybył do kliniki
Czas 8:58: Pacjent 5 wychodzi z gabinetu 1, czekal od 8:43 do 8:43
Czas 8:58: Pacjent 7 wchodzi do gabinetu 1 
Czas 9:07: Pacjent 6 wychodzi z gabinetu 2, czekal od 8:44 do 8:52
Czas 9:12: